This notebook cleans up the crosslink data for each NTD-nanobody pair, by first ensuring that the residue indices are numbered consistently, and then extracts crosslink sets for each nanobody-crosslink-pair in a (CSV) format suitable for integrative modeling. Note:

- NTD residues are reported according to the sequence of the full CoV2 spike protein. S1-NTD stretches from 16-305 of the S1 sequence. The structure used for NTD ([7LY3](https://www.rcsb.org/structure/7LY3), reported by Veesler et. al) had residues 72,73 and a stretch from 170-186 missing. These were "filled in" using MODELLER.
<br>

- nanobody sequences used in the crosslinking experiment have an additional N-terminal GS "tag" and a C-terminal LEHHHHHH tag which are not present in the core nanobody sequence used to obtain comparative models. So, crosslinked residues must be accordingly renumbered. 

First import a few necessary modules and set filenames, output locations, etc. 

In [2]:
import os
import numpy as np
import pandas as pd
from collections import namedtuple, OrderedDict
from Bio import SeqIO

FASTA_FN = "../spike_nanobody.fasta.txt"
XL_FN = "crosslinks_24Jun2021.xlsx"
OUTDIR = "xl"

RECEPTOR = "ntd"
LIGANDS = ["s1-49"]
RECEPTOR_OFFSET = -15
RECEPTOR_RANGE = (16, 305)

FASTA = {r.id: r.seq._data for r in SeqIO.parse(FASTA_FN, format="fasta")}

Raw crosslink data are kept in an input spreadsheet. While all of the links are manually curated, some may be undesirable to include in the final modeling. Any crosslink that has a value of 0 in the "included" column will be excluded while generating the formatted CSV files.

In [3]:
def _get_resid(peptide_seq, seq, offset=0):
    xl_loc = peptide_seq.find("*") - 1
    if xl_loc < 0:
        return None
    peptide_seq_cleaned = "".join([s for s in peptide_seq if s != "*"])
    out = 1 + seq.find(peptide_seq_cleaned) + xl_loc
    return out - offset

For good measure, the function below will perform additional checks to detect duplicate crosslinks and then set all linkers to DSS, since that is what is used.

In [4]:
def _trim_crosslinks(xls):
    d = OrderedDict()
    for xl in xls:
        p1, r1, p2, r2 = xl
        key = (p1, r1, p2, r2)
        if key not in d:
            d[key] = 1
        else:
            d[key] += 1
    
    xls_out = []
    for k, v in d.items():
        if v > 1:
            xl_str = "(%s, %d, %s, %d)" % k
            print("Duplicate found for XL %s, num_copies = %d" % (xl_str, v))
        xls_out.append(k)
    return xls_out 

In [7]:
def process_XL_set(ligand, outfn):
    receptor_seq = FASTA[RECEPTOR]
    ligand_seq = FASTA[ligand]
    df = pd.read_excel(XL_FN, sheet_name="%s_%s" % (RECEPTOR, ligand))
    xls = []
    for i in range(len(df)):
        row_num = i+2
        
        # ------------------------------------------
        # LIGAND (i.e. NANOBODY) CROSSLINKED RESIDUE
        # ------------------------------------------
        this_df = df.iloc[i]
        ligand_pep_seq = this_df["peptide1"]
        ligand_res = this_df["residue1"]
        ligand_res_computed = _get_resid(ligand_pep_seq, "GS" + ligand_seq) # add N-terminal "GS" tag
        # ignore if not found
        if ligand_res_computed is None:
            continue       
        # ignore if either residue on the N-terminal tag is crosslinked
        if ligand_res_computed <= 2:
            continue
        # check consistency with given residue number by collaborator
        if ligand_res != ligand_res_computed:
            print("Row %d. Ligand XL residue ID mismatch. Given: %d, computed from peptide seq: %d" % \
                 (row_num, ligand_res, ligand_res_computed))
        
        # when all checks have passesd, subtract the added tag
        ligand_res_computed -= 2
        
        
        # ---------------------------------------
        # RECEPTOR (i.e. RBD) CROSSLINKED RESIDUE
        # ---------------------------------------
        receptor_pep_seq = this_df["peptide2"]
        receptor_res = this_df["residue2"]
        receptor_res_computed = _get_resid(receptor_pep_seq, receptor_seq, RECEPTOR_OFFSET)
        # ignore if not found
        if receptor_res_computed is None:
            continue
        # ignore if outside structurally available receptor sequence
        if receptor_res_computed not in range(*RECEPTOR_RANGE):
            continue
        # check consistency with given residue number by collaborator
        if receptor_res != receptor_res_computed:
            print("Row %d. Receptor XL residue ID mismatch. Given: %d, computed from peptide seq: %d" % \
                 (row_num, receptor_res, receptor_res_computed))
        
        # if this crosslink has been explicitly excluded by collaborator, remove it
        if not this_df["included"]:
            continue
        
        # add the crosslink
        xl = (RECEPTOR, receptor_res_computed, ligand, ligand_res_computed)
        xls.append(xl)
    
    xls_trimmed = _trim_crosslinks(xls)
    out_df = pd.DataFrame(xls_trimmed, columns=["receptor", "receptor_residue", "ligand", "ligand_residue"])
    out_df.to_csv(outfn, index=False)
    display(out_df)

In [8]:
os.makedirs(OUTDIR, exist_ok=True)
for l in LIGANDS:
    print("Processing XLs for %s" % l)
    outfn = os.path.join(OUTDIR, "%s.csv" % l)
    process_XL_set(l, outfn)
    print("\n\n")

Processing XLs for s1-49


,receptor,receptor_residue,ligand,ligand_residue
0,ntd,187,s1-49,68
1,ntd,41,s1-49,68
2,ntd,182,s1-49,68
3,ntd,187,s1-49,47
4,ntd,97,s1-49,79
5,ntd,187,s1-49,79
6,ntd,182,s1-49,79
7,ntd,41,s1-49,79
